### **1.2.1 注意力机制实现的多尺度基准测试**
---

请编写一个脚本，在不同的尺度下对你实现的注意力机制进行基准测试。脚本应包含以下步骤：

*   **(a)** 将批大小（Batch Size）固定为 8，且不使用多头注意力机制（即：移除“头”维度，将其看作单头）。
*   **(b)** 遍历以下参数的笛卡尔积：头嵌入维度 $d_{model}$ 取值为 `[16, 32, 64, 128]`，序列长度（Sequence Length）取值为 `[256, 1024, 4096, 8192, 16384]`。
*   **(c)** 创建对应尺寸的随机输入张量 $Q, K, V$。
*   **(d)** 测量使用这些输入进行 100 次前向传播（Forward Passes）的时间。
*   **(e)** 在反向传播开始前，测量当前正在使用的显存量（Memory in use），并测量 100 次反向传播（Backward Passes）的时间。
*   **(f)** 确保进行预热（Warm up），并且在每次前向/反向传播之后调用 `torch.cuda.synchronize()`（以确保测速准确）。

**实验报告要求：**

汇报在这些配置下得到的耗时数据（如果报错则汇报显存溢出错误，即 Out-of-Memory, OOM）。
1.  在什么样的尺寸下会出现显存溢出（OOM）？
2.  针对你发现的出现 OOM 的最小配置之一，进行注意力机制显存占用的“账目计算”（你可以使用 Assignment 1 中关于 Transformer 显存占用的计算公式）。
3.  为了反向传播而保存（缓存）的显存量是如何随序列长度变化的？
4.  你会采取什么方法来消除这部分显存开销？

**交付物：**
一份包含耗时数据的表格、显存占用的计算推导过程，以及 1-2 段文字的回答。

---

In [6]:
import os
import torch.nn.functional as F

def pytorch_naive_attention(q, k, v, mask=None):
    """
    实现朴素的 PyTorch Attention: softmax(Q @ K.T / sqrt(dk)) @ V
    
    参数:
        q: Query 张量，形状为 (batch_size, seq_len, d_model)
        k: Key 张量，形状为 (batch_size, seq_len, d_model)
        v: Value 张量，形状为 (batch_size, seq_len, d_model)
        mask: 可选的掩码张量 (例如 causal mask)
        
    返回:
        output: 注意力计算后的输出，形状为 (batch_size, seq_len, d_model)
    """
    #step1:calculate attentionScore
    attentionScore=q@k.transpose(-2,-1)

    #step2:normalize
    d_k=q.shape[-1]
    attentionScore=attentionScore/d_k**0.5

    #step3:mask
    if mask is not None:
        attentionScore=attentionScore.masked_fill(~mask,float('-inf'))

    #step4:softmax
    attentionScore=torch.softmax(attentionScore,dim=-1)

    #step5:calculate output
    output=attentionScore@v
    return output

In [4]:
import time
import torch
import pandas as pd
import numpy as np

def benchmark_attention():
    batch_size = 8
    d_model_list = [16, 32, 64, 128]
    seq_len_list = [256, 1024, 4096, 8192, 16384]

    device = "cuda" if torch.cuda.is_available() else "cpu"
    results = []

    for d_model in d_model_list:
        for seq_len in seq_len_list:
            print(f"Testing: d_model={d_model}, seq_len={seq_len}...")
            
            # 清理显存缓存
            if device == "cuda":
                torch.cuda.empty_cache()
            
            try:
                # 构造数据，需要 requires_grad 来测试后向传播
                q = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)
                k = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)
                v = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)

                # --- 1. Warm up ---
                for _ in range(10):
                    _ = pytorch_naive_attention(q, k, v)

                # --- 2. Measure Forward Pass ---
                if device == "cuda": torch.cuda.synchronize()
                start_fwd = time.perf_counter()
                
                for _ in range(100):
                    out = pytorch_naive_attention(q, k, v)
                
                if device == "cuda": torch.cuda.synchronize()
                end_fwd = time.perf_counter()
                avg_fwd = (end_fwd - start_fwd) / 100 * 1000  # 转为 ms

                # --- 3. Measure Memory ---
                # 在后向传播开始前记录显存
                mem_usage = torch.cuda.memory_allocated(device) / (1024 ** 2) # MB

                # --- 4. Measure Backward Pass ---
                grad_output = torch.randn_like(out)
                if device == "cuda": torch.cuda.synchronize()
                start_bwd = time.perf_counter()
                
                for _ in range(100):
                    out.backward(grad_output, retain_graph=True)
                
                if device == "cuda": torch.cuda.synchronize()
                end_bwd = time.perf_counter()
                avg_bwd = (end_bwd - start_bwd) / 100 * 1000 # ms

                results.append({
                    "d_model": d_model,
                    "seq_len": seq_len,
                    "fwd_ms": f"{avg_fwd:.3f}",
                    "bwd_ms": f"{avg_bwd:.3f}",
                    "mem_mb": f"{mem_usage:.2f}"
                })

            except RuntimeError as e:
                if "out of memory" in str(e).lower():
                    print(f"OOM at seq_len={seq_len}")
                    results.append({
                        "d_model": d_model,
                        "seq_len": seq_len,
                        "fwd_ms": "OOM",
                        "bwd_ms": "OOM",
                        "mem_mb": "OOM"
                    })
                else:
                    raise e

    # 打印结果表格 (作业 1.1.2 建议使用 pandas 打印表格)
    df = pd.DataFrame(results)
    print("\nBenchmark Results:")
    print(df.to_markdown()) # 需要 pip install tabulate

if __name__ == "__main__":
    benchmark_attention()

Testing: d_model=16, seq_len=256...


/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:829: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:179.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Testing: d_model=16, seq_len=1024...
Testing: d_model=16, seq_len=4096...
Testing: d_model=16, seq_len=8192...
Testing: d_model=16, seq_len=16384...
OOM at seq_len=16384
Testing: d_model=32, seq_len=256...
Testing: d_model=32, seq_len=1024...
Testing: d_model=32, seq_len=4096...
Testing: d_model=32, seq_len=8192...
Testing: d_model=32, seq_len=16384...
OOM at seq_len=16384
Testing: d_model=64, seq_len=256...
Testing: d_model=64, seq_len=1024...
Testing: d_model=64, seq_len=4096...
Testing: d_model=64, seq_len=8192...
Testing: d_model=64, seq_len=16384...
OOM at seq_len=16384
Testing: d_model=128, seq_len=256...
Testing: d_model=128, seq_len=1024...
Testing: d_model=128, seq_len=4096...
Testing: d_model=128, seq_len=8192...
Testing: d_model=128, seq_len=16384...
OOM at seq_len=16384

Benchmark Results:
|    |   d_model |   seq_len | fwd_ms   | bwd_ms   | mem_mb   |
|---:|----------:|----------:|:---------|:---------|:---------|
|  0 |        16 |       256 | 0.118    | 6.040    | 10.62 


Benchmark Results:
|    |   d_model |   seq_len | fwd_ms   | bwd_ms   | mem_mb   |
|---:|----------:|----------:|:---------|:---------|:---------|
|  0 |        16 |       256 | 0.118    | 6.040    | 10.62    |
|  1 |        16 |      1024 | 1.241    | 2.177    | 50.38    |
|  2 |        16 |      4096 | 15.336   | 29.282   | 536.75   |
|  3 |        16 |      8192 | 64.601   | 120.728  | 2082.25  |
|  4 |        16 |     16384 | OOM      | OOM      | OOM      |
|  5 |        32 |       256 | 0.102    | 0.431    | 23.25    |
|  6 |        32 |      1024 | 1.241    | 2.184    | 52.50    |
|  7 |        32 |      4096 | 16.176   | 30.388   | 545.25   |
|  8 |        32 |      8192 | 67.603   | 128.251  | 2100.25  |
|  9 |        32 |     16384 | OOM      | OOM      | OOM      |
| 10 |        64 |       256 | 0.108    | 0.307    | 28.25    |
| 11 |        64 |      1024 | 1.306    | 2.512    | 56.75    |
| 12 |        64 |      4096 | 19.095   | 34.864   | 562.25   |
| 13 |        64 |      8192 | 81.289   | 152.397  | 2136.25  |
| 14 |        64 |     16384 | OOM      | OOM      | OOM      |
| 15 |       128 |       256 | 0.201    | 0.423    | 38.25    |
| 16 |       128 |      1024 | 1.807    | 3.657    | 65.25    |
| 17 |       128 |      4096 | 27.715   | 52.034   | 596.25   |
| 18 |       128 |      8192 | 127.961  | 238.462  | 2208.25  |
| 19 |       128 |     16384 | OOM      | OOM      | OOM      |

# 实验报告要求：

汇报在这些配置下得到的耗时数据（如果报错则汇报显存溢出错误，即 Out-of-Memory, OOM）。

## Q1：在什么样的尺寸下会出现显存溢出（OOM）？
针对你发现的出现 OOM 的最小配置之一，进行注意力机制显存占用的“账目计算”（你可以使用 Assignment 1 中关于 Transformer 显存占用的计算公式）。
A:当seq_len=16384的时候，不管d_model大小，内存都会溢出

Q,K,V Matrix=batchSize*d_model*seq_len*3*4byte=8*16*16384*3*4=0.02GB

Forward:attentionScore=batchsize*seq_len*seq_len*4byte=8*16384*16384*4=8GB

Backward:保留前向的矩阵8GB

total=QKVMatrix+Forward+Backward>16GB（GPU T4*2）


# 1.3.0

---

### **问题 (torch_compile)：2分**

**(a)** 扩展你之前的注意力机制基准测试脚本，加入你所实现的 PyTorch 注意力机制的**编译版本（compiled version）**。在与上述 `pytorch_attention` 问题（即 1.2.1 节）相同的配置下，对比其与非编译版本的性能。

*   **交付物**：一张对比表格，比较编译后的注意力模块与之前 `pytorch_attention` 问题中非编译版本的**前向（forward）**和**反向（backward）**传播耗时。

**(b)** 现在，在你之前的端到端基准测试脚本（即 1.1.3 节中用于测试整个模型的脚本）中，**编译你的整个 Transformer 模型**。前向传播的性能发生了怎样的变化？前向传播、反向传播以及优化器步骤（optimizer steps）组合在一起的整体性能又发生了怎样的变化？

*   **交付物**：一张对比表格，比较**原始原生（vanilla）** Transformer 模型与**编译后（compiled）** Transformer 模型的性能。

---

### **💡 任务重点说明：**

#### **1. 针对 (a) 的实现：**
你不需要写新函数，只需要利用 `torch.compile`。
*   **做法**：在测试循环开始前，执行 `compiled_fn = torch.compile(pytorch_naive_attention)`。
*   **意义**：你会观察到，由于 `torch.compile` 尝试将矩阵乘法、除法、Mask、Softmax 这些操作“融合”进更少的 GPU 内核（Kernels）中，减少了显存读写次数，速度应该会有提升。

#### **2. 针对 (b) 的实现：**
这次是针对**整个模型类**。
*   **做法**：`compiled_model = torch.compile(model)`。
*   **测量内容**：这次不仅仅测 Attention 函数，还要测量一个完整的训练步（Training Step），包括：
    1.  `output = compiled_model(input)`（前向）
    2.  `loss.backward()`（反向）
    3.  `optimizer.step()`（优化器更新）
*   **意义**：全模型编译通常比单函数编译有更大的优化空间，因为它能跨层进行算子融合。

#### **3. ⚠️ 实验陷阱提醒：**
*   **预热（Warm-up）**：`torch.compile` 在**第一次执行**时会非常慢，因为它在后台调用 Triton 生成内核代码。你必须在计时循环开始前，先跑几次模型，确保编译完成。
*   **显存**：注意观察，虽然速度变快了，但在长序列（如 16384）下，编译版是否依然会报错 OOM？（答案通常是：是的，因为它依然要存 $N^2$ 的注意力矩阵）。


In [8]:
import time
import torch
import pandas as pd
import numpy as np

def benchmark_attention():
    batch_size = 8
    d_model_list = [16, 32, 64, 128]
    seq_len_list = [256, 1024, 4096, 8192, 16384]
    
    if torch.cuda.is_available():
        device = "cuda"
        print(f"Is CUDA available: {torch.cuda.is_available()}")
        print(f"GPU Device Name: {torch.cuda.get_device_name(0)}")
        props = torch.cuda.get_device_properties(0)
        print(f"Total Memory: {props.total_memory / 1024**2:.2f} MB")
        print(f"Processor Count: {props.multi_processor_count}")
        print(f"Compute Capability: {props.major}.{props.minor}")
    else:
        device ="cpu"
    results = []

    compiled_naive_attention = torch.compile(pytorch_naive_attention)
    for d_model in d_model_list:
        for seq_len in seq_len_list:
            print(f"Testing: d_model={d_model}, seq_len={seq_len}...")
            
            # 清理显存缓存
            if device == "cuda":
                torch.cuda.empty_cache()
            
            try:
                # 构造数据，需要 requires_grad 来测试后向传播
                q = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)
                k = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)
                v = torch.randn(batch_size, seq_len, d_model, device=device, requires_grad=True)

                # --- 1. Warm up ---
                for _ in range(10):
                    warm_up_out = compiled_naive_attention(q, k, v)
                    warm_up_out.backward(torch.randn_like(warm_up_out))
                    
                # --- 2. Measure Forward Pass ---
                if device == "cuda": torch.cuda.synchronize()
                start_fwd = time.perf_counter()
                
                for _ in range(100):
                    out = compiled_naive_attention(q, k, v)
                
                if device == "cuda": torch.cuda.synchronize()
                end_fwd = time.perf_counter()
                avg_fwd = (end_fwd - start_fwd) / 100 * 1000  # 转为 ms

                # --- 3. Measure Memory ---
                # 在后向传播开始前记录显存
                mem_usage = torch.cuda.memory_allocated(device) / (1024 ** 2) # MB

                # --- 4. Measure Backward Pass ---
                grad_output = torch.randn_like(out)
                if device == "cuda": torch.cuda.synchronize()
                start = time.perf_counter()
                
                for _ in range(100):
                    out=compiled_naive_attention(q,k,v)
                    out.backward(grad_output)
                if device == "cuda": torch.cuda.synchronize()
                end = time.perf_counter()
                avg_fwd_bwd = (end - start) / 100 * 1000 # ms 注意这里包含前向和反向的时间
                real_bwd_ms=avg_fwd_bwd-avg_fwd

                results.append({
                    "d_model": d_model,
                    "seq_len": seq_len,
                    "fwd_ms": f"{avg_fwd:.3f}",
                    "bwd_ms": f"{real_bwd_ms:.3f}",
                    "mem_mb": f"{mem_usage:.2f}"
                })

            except RuntimeError as e:
                if "out of memory" in str(e).lower():
                    print(f"OOM at seq_len={seq_len}")
                    results.append({
                        "d_model": d_model,
                        "seq_len": seq_len,
                        "fwd_ms": "OOM",
                        "bwd_ms": "OOM",
                        "mem_mb": "OOM"
                    })
                else:
                    raise e

    # 打印结果表格 (作业 1.3.1 建议使用 pandas 打印表格)
    df = pd.DataFrame(results)
    print("\nBenchmark Results:")
    print(df.to_markdown()) # 需要 pip install tabulate

if __name__ == "__main__":
    benchmark_attention()

Is CUDA available: True
GPU Device Name: Tesla T4
Total Memory: 14912.69 MB
Processor Count: 40
Compute Capability: 7.5
Testing: d_model=16, seq_len=256...
Testing: d_model=16, seq_len=1024...
Testing: d_model=16, seq_len=4096...
Testing: d_model=16, seq_len=8192...
Testing: d_model=16, seq_len=16384...
OOM at seq_len=16384
Testing: d_model=32, seq_len=256...
Testing: d_model=32, seq_len=1024...
Testing: d_model=32, seq_len=4096...
Testing: d_model=32, seq_len=8192...
Testing: d_model=32, seq_len=16384...
OOM at seq_len=16384
Testing: d_model=64, seq_len=256...
Testing: d_model=64, seq_len=1024...
Testing: d_model=64, seq_len=4096...
Testing: d_model=64, seq_len=8192...
Testing: d_model=64, seq_len=16384...
OOM at seq_len=16384
Testing: d_model=128, seq_len=256...
Testing: d_model=128, seq_len=1024...
Testing: d_model=128, seq_len=4096...
Testing: d_model=128, seq_len=8192...
Testing: d_model=128, seq_len=16384...
OOM at seq_len=16384

Benchmark Results:
|    |   d_model |   seq_len | 

- Is CUDA available: True
- GPU Device Name: Tesla T4
- Total Memory: 14912.69 MB
- Processor Count: 40
- Compute Capability: 7.5

before complie:
|    |   d_model |   seq_len | fwd_ms   | bwd_ms   | mem_mb   |
|---:|----------:|----------:|:---------|:---------|:---------|
|  0 |        16 |       256 | 0.118    | 6.040    | 10.62    |
|  1 |        16 |      1024 | 1.241    | 2.177    | 50.38    |
|  2 |        16 |      4096 | 15.336   | 29.282   | 536.75   |
|  3 |        16 |      8192 | 64.601   | 120.728  | 2082.25  |
|  4 |        16 |     16384 | OOM      | OOM      | OOM      |
|  5 |        32 |       256 | 0.102    | 0.431    | 23.25    |
|  6 |        32 |      1024 | 1.241    | 2.184    | 52.50    |
|  7 |        32 |      4096 | 16.176   | 30.388   | 545.25   |
|  8 |        32 |      8192 | 67.603   | 128.251  | 2100.25  |
|  9 |        32 |     16384 | OOM      | OOM      | OOM      |
| 10 |        64 |       256 | 0.108    | 0.307    | 28.25    |
| 11 |        64 |      1024 | 1.306    | 2.512    | 56.75    |
| 12 |        64 |      4096 | 19.095   | 34.864   | 562.25   |
| 13 |        64 |      8192 | 81.289   | 152.397  | 2136.25  |
| 14 |        64 |     16384 | OOM      | OOM      | OOM      |
| 15 |       128 |       256 | 0.201    | 0.423    | 38.25    |
| 16 |       128 |      1024 | 1.807    | 3.657    | 65.25    |
| 17 |       128 |      4096 | 27.715   | 52.034   | 596.25   |
| 18 |       128 |      8192 | 127.961  | 238.462  | 2208.25  |
| 19 |       128 |     16384 | OOM      | OOM      | OOM      |


after compile:
|    |   d_model |   seq_len | fwd_ms   | bwd_ms   | mem_mb   |
|---:|----------:|----------:|:---------|:---------|:---------|
|  0 |        16 |       256 | 0.354    | 0.487    | 19.25    |
|  1 |        16 |      1024 | 1.315    | 1.227    | 52.38    |
|  2 |        16 |      4096 | 15.356   | 23.546   | 544.75   |
|  3 |        16 |      8192 | 65.147   | 98.962   | 2098.25  |
|  4 |        16 |     16384 | OOM      | OOM      | OOM      |
|  5 |        32 |       256 | 0.939    | 0.860    | 24.25    |
|  6 |        32 |      1024 | 3.713    | 4.169    | 56.50    |
|  7 |        32 |      4096 | 23.691   | 32.629   | 561.25   |
|  8 |        32 |      8192 | 80.472   | 124.365  | 2132.25  |
|  9 |        32 |     16384 | OOM      | OOM      | OOM      |
| 10 |        64 |       256 | 0.969    | 0.878    | 30.25    |
| 11 |        64 |      1024 | 3.990    | 4.636    | 64.75    |
| 12 |        64 |      4096 | 28.613   | 41.088   | 594.25   |
| 13 |        64 |      8192 | 102.531  | 168.196  | 2200.25  |
| 14 |        64 |     16384 | OOM      | OOM      | OOM      |
| 15 |       128 |       256 | 1.091    | 0.925    | 42.25    |
| 16 |       128 |      1024 | 4.598    | 5.939    | 81.25    |
| 17 |       128 |      4096 | 40.429   | 60.686   | 660.25   |
| 18 |       128 |      8192 | 147.319  | 250.266  | 2336.25  |
| 19 |       128 |     16384 | OOM      | OOM      | OOM      |

### 1.3.1 Example - Weighted Sum (Triton 入门)
---
**它是做什么的：**
如何用 Triton 编写一个简单的内核：计算矩阵 $X$ 每一行与向量 $w$ 的点积。

**为什么要先看这个：**
*   **理解“分块指针” (Block Pointers)**：这是 Triton 的核心。以前写 CUDA 算索引很痛苦（要算 `threadIdx`, `blockIdx` 等），Triton 使用 `tl.make_block_ptr` 让你像操作切片一样操作 GPU 内存。
*   **理解“步进” (Advance)**：学习如何用 `ptr.advance` 在内存中移动你的窗口。
*   **代码封装**：学习如何把 Triton 内核包装进 `torch.autograd.Function` 的 `forward` 和 `backward` 中，使其能无缝融入 PyTorch 的训练流程。

**你应该怎么做：**
1.  **不要跳过阅读**：仔细阅读 PDF 第 10-14 页的代码。
2.  **在 Kaggle 手敲一遍**：尝试运行 PDF 中提供的 `weighted_sum` 代码。
3.  **理解反向传播**：看懂梯度 $dw$ 和 $dx$ 是如何在 Triton 循环里累加的。

---

In [ ]:
#1.3.1 Example - Weighted Sum
def weighted_sum(x,weight):
    # 此处假设 x 的维度是 [..., D]，weight 的维度是 [D],维度不匹配， 广播+点积
    return (weight *x).sum(axis=-1)


import triton
import triton.language as tl
@triton.jit
def weight_sum_fwd(
    x_ptr,weight_ptr, #输入指针
    output_ptr, #输出指针
    x_row_stride,x_stride_dim,#步长，告诉我们如何在张量的每个轴上移动一个元素
    weight_stride_dim,#通常为1
    output_stride_row,#通常为1
    ROWS,D,#矩阵总行数和维度
    ROWS_TILE_SIZE:tl.constexpr,
    D_TILE_SIZE: tl.constexpr,#分块大小：必须在编译时已知 对于3*2的矩阵ROWS_TILE_SIZE=1，D_TILE_SIZE=2
):
    #每个实例将计算一组行分块的加权和
    # tl.program_id(0) 指示当前在哪个线程块
    row_tile_idx=tl.program_id(0)
    
    #块指针（Block pointers） 让我们能选择内存中的一个N维区域并移动它
    """
    块指针需要知道：
    - 指向张量的第一个元素的指针
    - 张量的总形状（以处理越界访问）
    - 每个维度的步长（以正确使用内存布局）
    - 起始块的N维坐标(offsets)
    - 每次加载/存储时使用的块形状
    - 维度在内存中的排列顺序
    """
    x_block_ptr=tl.make_block_ptr(
        x_ptr,
        shape=(ROWS,D,),
        strides=(x_row_stride,x_stride_dim),
        offsets=(row_tile_idx * ROWS_TILE_SIZE,0),
        block_shape=(ROWS_TILE_SIZE,D_TILE_SIZE),
        order=(1,0),
    )

    weight_block_ptr=tl.make_block_ptr(
        weight_ptr,
        shape=(D,),
        strides=(weight_stride_dim,),
        offsets=(0,),
        block_shape=(D_TILE_SIZE,),
        orser=(0,),
    )

    output_block_ptr=tl.make_block_ptr(
        output_ptr,
        shape=(ROWS,),
        strides=(output_stride_row,),
        offsets=(row_tile_idx * ROWS_TILE_SIZE,),
        block_shape=(ROWS_TILE_SIZE,),
        orser=(0,),
    )
    #初始化一个缓冲区用于写入结果
    output=tl.zeros((ROWS_TILE_SIZE,),dtype=tl.float32)
    #在D维度上进行循环
    for i in range(tl.cdiv(D,D_TILE_SIZE)):
        #加载当前的块指针内容并带有边界检查
        row=tl.load(x_block_ptr,boundary_check=(0,1),padding_option="zero")
        weight=tl.load(weight_block_ptr,boundary_check=(0,),padding_option="zero")

        #计算该行分块的加权和
        output+=tl.sum(row*weight[None,:],axis=1)

        #将指针移动到下一个分块
        x_block_ptr=x_block_ptr.advance((0,D_TILE_SIZE))
        weight_block_ptr=weight_block_ptr.advance((D_TILE_SIZE,))
    #将结果写回输出快指针
    tl.store(output_block_ptr,output,boundary_check=(0,))

class WeightedSumFunc(torch.autograd.Function):
    @staticmethod
    def forward(ctx,x,weight):
        #x=(Batch, Seq_len, D) such as (8, 512, 128)
        D,output_dims=x.shape[-1],x.shape[:-1]
        input_shape=x.shape
        # 把前面所有的维度（Batch, Seq_len）全部压扁成一个维度。(8, 512, 128) 会变成 (4096, 128),这样我们的 Triton 内核只需要处理一个 2D 矩阵 就行了
        x=rearrange(x,"...d ->(...)d")
        """
        这一步将展平后的 x 和 weight 保存到 ctx 对象中。
        当你之后运行 loss.backward() 时,PyTorch 会把这两个张量传给 backward 函数，用来计算梯度。
        """
        ctx.save_for_backward(x,weight)

        assert len(weight.shape)==1 and weight.shape[0]==D,"Dimension mismatch"
        assert x.is_cuda and weight.is_cuda,"Expected CUDA tensor"
        assert x.is_contiguous(),"Our pointer arithmetic will assume contiguous x" #is_contiguous (连续性)：这是最关键的！Triton 的指针算法基于步长（Stride）。如果张量在内存里是不连续的（比如经过了转置但没调 contiguous()），指针计算就会乱掉，读到错误的数据。

        ctx.D_TILE_SIZE=triton.next_power_of_2(D)#Triton 的 block_shape 必须是 2 的幂,如果D=128，就会计算出128/16=8
        ctx.ROWS_TILE_SIZE=16
        ctx.input_shape=input_shape

        y=torch.empty(output_dims,device=x.device)

        n_rows=y.numel()

        """
        grid 定义：(cdiv(n_rows, ctx.ROWS_TILE_SIZE), )。这决定了启动多少个线程块。如果总共有 160 行，每块处理 16 行，就启动 10 个线程块。
        传递步长 (stride)：手动把 PyTorch 张量的内存步长传给内核。这就是我们在内核里计算地址的依据。
        """
        weight_sum_fwd[(tl.cdiv(n_rows,ctx.ROWS_TILE_SIZE),)](
            x.weight,
            y,
            x.stride(0),x.stride(1),
            weight.stride(0),
            y.stride(0),
            ROWS=n_rows,D=D,
            ROWS_TILE_SIZE=ctx.ROWS_TILE_SIZE,D_TILE_SIZE=ctx.D_TILE_SIZE,
        )
        return y.view(input_shape[:-1])#view把它还原成原始的形状 test
    
    

### 1.3.2 FlashAttention-2 Forward Pass (正式挑战)
这是整个作业中分值最高的部分之一。目标是实现 **Algorithm 1 (第 18 页)**。

它分成了两步走：

#### (a) 编写纯 PyTorch 的 Tiled Attention (Pure PyTorch, No Triton)
*   **任务**：按照 Algorithm 1 的逻辑，用普通的 Python `for` 循环和 PyTorch 切片来实现 Flash Attention。
*   **核心挑战：Online Softmax**。
    *   标准的 Softmax 需要知道一整行的最大值。
    *   Flash Attention 是分块算的。当你算到第 2 块时，发现第 2 块里有更大的值，你需要用公式去**修正**（Rescale）第 1 块已经算好的部分结果。
*   **公式（灵魂）**：
    *   $m_i^{(new)} = \max(m_i^{(old)}, \text{rowmax}(S_i^{(j)}))$
    *   $l_i^{(new)} = e^{m_i^{(old)} - m_i^{(new)}} \cdot l_i^{(old)} + \text{rowsum}(e^{S_i^{(j)} - m_i^{(new)}})$

#### (b) 编写 Triton 内核版
*   **任务**：把你刚才在 (a) 中写通的逻辑，翻译成 Triton 语法。
*   **优势**：Triton 版会比 (a) 快几百倍，因为它把所有操作（矩阵乘法、归一化、加权求和）全部融合在一个内核里。

---